In [ ]:
#import os
#import sys
#from pathlib import Path
#sys.path.append(str(Path(__file__).parent.parent))


In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from alns.rl.hybrid_agent import HybridAgent
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sp

from experiment_storage.file_paths import FilePaths
from experiment_storage.storage import EvaluationStorage


all_exp_ids = ['C101200_varinstsizeTSP', 'R101200_varinstsizeTSP', 'O101200_varinstsizeTSP',
              'C101200_varinstsizeCVRP', 'R101200_varinstsizeCVRP', 'O101200_varinstsizeCVRP',
              'C101200_varinstsizeVRPTW', 'R101200_varinstsizeVRPTW', 'O101200_varinstsizeVRPTW',
              'C101200_varinstsizeHVRP', 'R101200_varinstsizeHVRP', 'O101200_varinstsizeHVRP',
              'C101200_varinstsizeLRP', 'R101200_varinstsizeLRP', 'O101200_varinstsizeLRP']

# all_exp_ids = ['O101200_varinstsizeTSP',
#               'O101200_varinstsizeCVRP',
#               'O101200_varinstsizeVRPTW',
#               'O101200_varinstsizeHVRP',
#               'O101200_varinstsizeLRP']

parent_dir = "/Users/vdarvariu/experiment_data/ates"
#parent_dir = '/Users/shunee/github/ATES2022/experiment_data'

fp_out = FilePaths(parent_dir, "aggregate", setup_directories=True)

# filter_agent = [HybridAgent.algorithm_name]
filter_agent = None


# Final test set results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for exp_id in all_exp_ids:
    fp_in = FilePaths(parent_dir, exp_id, setup_directories=False)
    storage = EvaluationStorage(fp_in)
    param_spaces, df = storage.get_hyperparameter_optimisation_data(exp_id, False, task_type="tune", fp_in=fp_in)
    df = df.drop(columns=["network_generator", "objective_function", "graph_id"])
    df = df.drop(df[df.agent_name != "dqnprob"].index)

    df['temp'] = df['hyperparams_id'].apply(lambda x: param_spaces["total_objective"]["dqnprob"][str(x)]["alns_temp"])

    sns.set(font_scale=2.5)
    plt.rcParams["lines.linewidth"] = 4
    plt.rc('font', family='serif')

    fig, ax = plt.subplots(1, 1, squeeze=True, figsize=(10,10))
    # ax.set_xscale('log')
    sns.lineplot(data=df, x="temp", y="avg_perf", ax=ax)

    maxes = (df.groupby(["temp"])["avg_perf"].max())
    maxes_x = maxes.index.tolist()
    maxes_y = maxes.values
    ax.scatter(maxes_x, maxes_y)

    plt.savefig(fp_out.figures_dir / f"tune_{exp_id}.pdf", bbox_inches="tight")
    ax.set_title(exp_id)
    # print(param_spaces)

